<a href="https://colab.research.google.com/github/tytttyyyttyy/net_learning/blob/main/auto_cer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread PyPDF2

In [ ]:
!pip install gspread


In [ ]:
import pandas as pd
from google.colab import drive
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from PyPDF2 import PdfReader, PdfWriter
from jinja2 import Template

from google.colab import auth
auth.authenticate_user()

import google.auth
from google.auth.transport.requests import Request
from google.colab import auth
from oauth2client.client import GoogleCredentials
from gspread import authorize

creds, _ = google.auth.default()
gc = authorize(creds)

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/auto certificate/HR_Dataset_2024.csv'
df = pd.read_csv(file_path)

df['Emp #'] = df['Emp #'].astype(str)# 將 Emp # 欄位轉換為文字格式
emp_num = input("請輸入要查詢的Emp #：")# 讓使用者輸入特定Emp #
df_filtered = df[df['Emp #'] == emp_num]# 篩選出符合條件的資料


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
請輸入要查詢的Emp #：E0005


In [ ]:

# 輸出指定欄位
if len(df_filtered) > 0:
  #print(df_filtered[['Emp #', 'Employee Name', 'Location', 'National ID','Department','DOB', 'Date of Hire', 'Date of Termination']])
  # 取得員工資料
  employee_data = df_filtered[['Emp #', 'Employee Name', 'Location', 'National ID','Department','DOB', 'Date of Hire', 'Date of Termination']].to_dict(orient='records')[0]
else:
  print(f"查無Emp # {emp_num} 的資料")

In [ ]:
# 判斷是否有資料
if len(df_filtered) > 0:
    # 取得員工資料
    employee_data = df_filtered[['Emp #', 'Employee Name', 'Location', 'National ID','Department','DOB', 'Date of Hire', 'Date of Termination']].to_dict(orient='records')[0]

    # 將 NaN 轉換為字串 'nan'
    for key, value in employee_data.items():
        if pd.isna(value):
            employee_data[key] = 'nan'

    # 輸出員工資料
    print(employee_data)
else:
    print(f"查無Emp # {emp_num} 的資料")

# 將DataFrame轉換為二維列表
data = pd.DataFrame([employee_data]).values.tolist()[0]
data

{'Emp #': 'E0005', 'Employee Name': 'Singh, Nan ', 'Location': 'TWN', 'National ID': 1307059817, 'Department': 'Admin Offices', 'DOB': '1988/5/19', 'Date of Hire': '2015/5/1', 'Date of Termination': 'nan'}


['E0005',
 'Singh, Nan ',
 'TWN',
 1307059817,
 'Admin Offices',
 '1988/5/19',
 '2015/5/1',
 'nan']

In [ ]:
spreadsheet_id = '1poMhLbeQSrMNnJgTW5zS1bi_hqf6PnHnEi_2aTJ8kRc'
sheet_name = 'record'
# 取得 Google Spreadsheets 工作表
sheet = gc.open_by_key(spreadsheet_id).worksheet(sheet_name)

In [ ]:
sheet.append_row(data)